In [137]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [138]:
# !pip install folium

In [139]:
import json
import folium
import pandas as pd


In [140]:
path = '/content/drive/MyDrive/Colab/SeoulBigData/유동인구/'
json_file = 'HangJeongDong_ver20210701.geojson'

load_json = json.load(open(path + json_file, encoding='utf-8'))

df = pd.read_csv(path + 'top_all_ratio.csv', encoding='euc-kr')
df = df[:100]
# load_json

In [141]:
dong_ratio = 'dong_ratio2.csv'
dongData = pd.read_csv(path + dong_ratio, encoding='euc-kr')
dongData.head()
# dongData['ratio'] = dongData['ratio'] * 1000

,dong,ratio
0,가락1동,0.000222
1,가락2동,0.001640
2,가락본동,0.002737
3,가리봉동,0.002607
4,가산동,0.001429


In [142]:
seoul_map = folium.Map(location=[37.558079, 126.907864], zoom_start=9.5)

seoul_map.choropleth(geo_data=load_json,
                     data=dongData,
                     columns=['dong', 'ratio'],
                     fill_color='PuRd',
                     key_on='properties.adm_nm',
                     highlight=True,
                     fill_opacity=0.5, line_opacity=1,
                     legend_name='population ratio')

/usr/local/lib/python3.7/dist-packages/folium/folium.py:426: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [143]:
from folium.plugins import MarkerCluster

y = df.lat.values
x = df.lon.values
locations = list(zip(y,x))
popups = list(df.address.values)

icon_create_function = """\
function(cluster) {
    return L.divIcon({
    html: '<b>' + cluster.getChildCount() + '</b>',
    className: 'marker-cluster marker-cluster-large',
    iconSize: new L.Point(30, 30)
    });
}"""



map_mw2 = folium.Map(location=[37.557376, 126.904376], zoom_start = 16)  #망원시장 중심

marker_cluster = MarkerCluster(
    locations=locations, popups=popups,
    name='Mangwon',
    overlay=True,
    control=True,
    icon_create_function=icon_create_function
)

marker_cluster.add_to(map_mw2)
folium.LayerControl().add_to(map_mw2)


for index, row in df.iterrows():
    if index <= 19.8:
        col = 'grey'
    elif  19.8<index<=19.874: 
        
        col = 'yellow'
        
    elif 19.874 <index<=20.0237092:
        col = 'green'
    
    elif 20.0237092 <index<=20.3339816:
        col = 'blue'

    else:
        col = 'red'
        
    geoData = [row['lat'], row['lon']]
    folium.CircleMarker(geoData, radius=700*row['ratio'],color=col,fill_color=col, popup=row['address'], fill = True).add_to(seoul_map)

In [144]:
seoul_map